In [ ]:
# installing required packages for the current runtime

!pip install wget
!pip install jsonlines


In [ ]:
# importing required packages

import tweepy
import jsonlines
import json
import csv
import wget
import pandas as pd


In [ ]:
# user keys and tokens

consumer_key = ''
consumer_secret_key = ''
access_token = ''
access_token_secret = ''


In [ ]:
def process_media(entities):
  """accepts a tweet entities object,
  returns the number of images in the tweet"""

  ret = 0

  if entities.get('media'):
    ret += len(entities.get('media'))

  return ret if ret else None


In [ ]:
def get_all_tweets(user_handle):
  """accepts a user screen_name,
  returns a list of tweepy status objects"""

  # user authorization
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
  auth.set_access_token(access_token, access_token_secret)
  api = tweepy.API(auth)

  # loading tweets in batches of size of 200
  all_tweets = []
  new_tweets = api.user_timeline(screen_name = user_handle, count = 200)
  all_tweets.extend(new_tweets)
  oldest_tweet = all_tweets[-1].id - 1
  while len(new_tweets) > 0:
    new_tweets = api.user_timeline(screen_name = user_handle, count = 200, max_id = oldest_tweet)
    all_tweets.extend(new_tweets)
    oldest_tweet = all_tweets[-1].id - 1

  return all_tweets


In [ ]:
if __name__ == '__main__':

  # fetching all the tweets of a given user
  all_tweets = get_all_tweets(input("Enter handle : "))

  # writing all the tweets to jsonlines file
  writer = jsonlines.open('tweets.jsonl', mode = 'w')
  for tweet in all_tweets:
    writer.write(tweet._json)


In [ ]:
# extracting required information to a list of lists

# required columns
columns = ['id', 'created_at', 'text', 'retweet_count', 'favorite_count', 'entities']

data = []

with jsonlines.open('tweets.jsonl') as reader:
   for obj in reader:
     selected_row = []
     for item in columns:
       selected_row.append(obj[item])
     data.append(selected_row)


In [ ]:
# converting the list into a Dataframe with required columns

table = pd.DataFrame(data, columns = columns)
table.head(10)


In [ ]:
# extracting images from entities objects

table['image_count'] = table['entities'].apply(lambda x : process_media(x))
table.head(10)


In [ ]:
# removing the useless columns and printing the table

table.drop(['entities'], axis = 1, inplace = True)
table.head(table.shape[0])
